In [4]:
#####actual program huh
#####importing
import os
from IPython.display import Image, display
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Conv1D,MaxPooling2D
import matplotlib.pyplot as plt

#looking for an path in Google drive
train_path = '/content/drive/My Drive/Colab Notebooks/chest_xray11/train'
val_path = '/content/drive/My Drive/Colab Notebooks/chest_xray11/val'
test_path = '/content/drive/My Drive/Colab Notebooks/chest_xray11/test'

test_norm = ImageDataGenerator(rescale = 1./255)  #Image normalization

#transofrming images into matrices to work with; batch size = 30
train_batch = test_norm.flow_from_directory(train_path,target_size = (500,500),classes = ['NORMAL','PNEUMONIA'],batch_size=30)
test_batch = test_norm.flow_from_directory(test_path,target_size = (500,500),classes = ['NORMAL','PNEUMONIA'],batch_size=30)
val_batch = test_norm.flow_from_directory(val_path,target_size = (500,500),classes = ['NORMAL','PNEUMONIA'],batch_size=30)

#neural network model building
classifier = Sequential()

classifier.add(Conv2D(80, (3,3), input_shape=(500, 500, 3), activation='relu', strides= 2))
classifier.add(MaxPooling2D(pool_size=(3,3)))
classifier.add(Conv2D(80, (3,3), activation='relu', strides= 2))
classifier.add(MaxPooling2D(pool_size=(3,3)))

classifier.add(Flatten())

#classifier.add(Dense(50, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(2, activation='softmax'))
classifier.summary()

#network compiling
classifier.compile(loss = keras.losses.binary_crossentropy, optimizer = 'adam',metrics=['accuracy'])

#it is stored in history, since it works out for graphics
h = classifier.fit(train_batch,epochs = 1, steps_per_epoch=7,validation_data=val_batch,validation_steps=3)



#printing overall accuracy

#not sure whether it should be history or classifier here
test_accu = classifier.evaluate_generator(test_batch,steps=10)
print("Test accuracy is: ", test_accu[1]*100, '%' )



#graphs 
'''
#accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training set', 'Validation set'], loc='upper left')
plt.show()

#loss
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training set', 'Test set'], loc='upper left')
plt.show()
'''


###in this part I try to figure out recall, precision and f1

#getting predictions and reshaping
pred = classifier.predict(test_batch)
pred.shape
p = pred.T.reshape(-1, 1)

#making predictions either 0 or 1
pred_final = np.where(p>0.5,1,0)
pred_final 


#getting y_test once more, but dividing it into normal/ pneumonia (or 1/0) this time
labels = ['NORMAL', 'PNEUMONIA']   

def create_training_data(data_dir):              #creating the training data
    
    images = []
    
    for label in labels:
        dir = os.path.join(data_dir,label)
        class_num = labels.index(label)
        
        

        for image in os.listdir(dir):    
            
            
            images.append([class_num])
            
    return np.array(images)


y_test = create_training_data('/content/drive/My Drive/Colab Notebooks/chest_xray11/test')

#here we have different dimensions for goodness knows what reason
print(y_test.shape)
print(pred_final.shape)

#function on getting recall, precision and f1
def perf_measure(test_batch, pred_final):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(pred_final)): 
        if test_batch[i]==pred_final[i]==1:
           TP += 1
        if test_batch[i]==1 and test_batch[i]!=pred_final[i]:
           FP += 1
        if test_batch[i]==pred_final[i]==0:
           TN += 1
        if test_batch[i]==0 and test_batch[i]!=pred_final[i]:
           FN += 1

    return(TP, FP, TN, FN)



#function use and getting actual numbers

tp, fp, tn ,fn = perf_measure(y_test,pred_final)

precision = tp/(tp+fp)
recall = tp/(tp+fn)
f_score = (2*precision*recall)/(precision+recall)

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))
print("F-Score is {:.2f}".format(f_score))


Using TensorFlow backend.


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 249, 249, 80)      2240      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 83, 83, 80)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 80)        57680     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 80)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13520)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1730688 

IndexError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
